In [2]:
import qctoolkit as qtk
from matplotlib import pyplot as plt
import numpy as np
from copy import deepcopy
from glob import glob
import os
import itertools

#Note inline and notebook is contradicting to each other
%matplotlib inline

In [4]:
qmsetting_production = {
    'program': 'gaussian',
    'basis_set': 'cc-pVDZ',
    'theory': 'pbe',
    'threads': 1,
}
    
with open('ccpvdz_ne_gbasis.txt') as f:
    ne_basis = f.readlines()
    
qmsetting_pbe_same_basis = {
    'program': 'gaussian',
    #'basis_set': ar_basis, gaussian failed too many simple basis function?
    'basis_set': ne_basis,
    'theory': 'pbe',
    'threads': 1,
}

In [27]:
#Z_list = range(1, 14)
Z_list = range(1, 8)

def FD_diff_1D(qmsetting, base=[7,7,1.1], dz=0.05, get_mols=False, R=1.1):
    FD_dZ = [[base[0]-dz, base[1]+dz], False, [base[0]+dz, base[1]-dz]]
    mols = []
    inps = []
    for z, Z in enumerate(Z_list):
        for i, fd_dz in enumerate(FD_dZ):
            mol = qtk.Molecule()
            mol.build([[Z, 0, 0, 0], [14 - Z, R, 0, 0]])
            mol.name = 'diff_' + mol.name + '_%d' % i
            if (base[0] + base[1]) % 2 != 0:
                mol.setChargeMultiplicity(0, 2)
            inp = qtk.QMInp(mol, nuclear_charges=fd_dz, **qmsetting)
                
            inps.append(inp)
            mols.append(mol)
    if get_mols:
        return inps, mols
    else:
        return inps

In [28]:
inps = FD_diff_1D(qmsetting_pbe_same_basis)

In [29]:
qtk.qmWriteAll(inps, 'gaussian_N2_1D', compress=True, overwrite=True)

overwrite existing folder gaussian_N2_1D
qmWriteAll: compression completed 
